# CBAM : Mantık ve Öğrenme Sezgisi (Görüntü İşleme)

Bu defterde CBAM’i **matematiğe girmeden** ama **derin** şekilde açıklıyoruz:

- CBAM “nerenin önemli olduğunu” nasıl öğreniyor?
- Kanal dikkat (Channel Attention) ile uzamsal dikkat (Spatial Attention) neyi çözüyor?
- Eğitim (loss/gradient) bu maskeleri nasıl şekillendiriyor?
- Detection / Segmentation / Classification için pratik yerleştirme mantığı

Bu defter, “CBAM çalışıyor ama neden çalışıyor?” sorusuna odaklanır.


## 1) CBAM aslında ne yapıyor?

CBAM iki aşamalı bir filtre gibi çalışır:

1) **Channel Attention (CA)**: “Hangi feature türleri işime yarıyor?”  
2) **Spatial Attention (SA)**: “Bu faydalı feature’lar görüntünün neresinde önemli?”

CBAM, **aynı anda** değil **ardışık** şekilde uygular:
- Önce kanalı süzer
- Sonra konumu süzer

Bu yüzden CBAM, SE/ECA’ya göre daha güçlü ama daha maliyetli olabilir.


## 2) “Nerenin önemli olduğunu” nasıl anlıyor?

En kritik nokta şu:

CBAM **bir şey “anlamaz”**.  
CBAM **öğrenir**.

Elinde sadece iki şey vardır:
- CNN’in ara katman feature’ları (x)
- Eğitim sırasında gelen hata sinyali: loss → gradient

Yani CBAM, “şu bölge nesne” diye bir etiket görmez.  
Şunu öğrenir:

> “Bu bölgeyi güçlendirince model daha az hata yapıyor mu?”  
> “Şu bölgeyi baskılayınca model daha iyi mi oluyor?”

Bu tamamen deneme-yanılma + gradient güncellemesiyle oturur.


## 3) Spatial Attention (SA) kısmı neye bakıyor?

Spatial Attention “pikselleri” doğrudan sınıflandırmıyor.  
Şu soruyu soruyor:

> “Şu konumda (x,y) güçlü ve anlamlı bir sinyal var mı?”

SA bunu pratikte şöyle yapar:
- Kanalların hepsine tek tek bakmak yerine, kanalları özetleyen iki harita çıkarır:
  - Ortalama harita: “Genel aktivasyon burada yüksek mi?”
  - Maksimum harita: “En az bir kanal burada çok mu güçlü?”

Sonra bu iki haritayı birleştirip küçük bir konvolüsyondan geçirir.
Bu konvolüsyon şunu öğrenir:

- Gürültüyü bastır
- Tutarlı ve yapısal bölgeleri öne çıkar
- Komşu piksellerden bağlam al (tek piksele takılma)

Sonuç: bir “mask” çıkar ve feature’ı o maske ile çarpar.


## 4) “Mask” neden işe yarıyor?

Mask şunu yapar:

- Modelin “zaten güçlü sinyal var” dediği yerleri biraz daha belirginleştirir
- Modelin sürekli hata yaptığı, alakasız aktivasyon üreten yerleri baskılar

Bu, “modelin dikkatini” fiziksel olarak oraya taşımak değil.  
Daha doğru ifade:

> Modelin downstream katmanlara taşıdığı bilgiyi düzenlemek.

Yani CBAM, feature taşıma hattında “gain control” gibi davranır.


## 5) CBAM’in öğrenme mekanizması (matematik yok)

CBAM’in nasıl öğrendiğini şu zincirle düşün:

1) Model bir görüntü görür  
2) CBAM bir maske üretir (ilk başta rastgele/orta karar)  
3) Model tahmin yapar  
4) Tahmin yanlışsa loss artar  
5) Geri yayılım (backprop) maske üreten ağırlıklara “şu yönde değiş” der  
6) Birçok örnekten sonra maske şu davranışı öğrenir:
   - Yararlı bölgeleri aç
   - Zararlı bölgeleri kapat

Bu yüzden “önem” kavramı görev bağımlıdır:
- Detection’da başka bölgeler önemli olur
- Segmentation’da sınırlar önemli olabilir
- Classification’da bazen sadece global ipuçları önemlidir


## 6) Kanal Dikkat (CA) ile Uzamsal Dikkat (SA) farkı

- **CA**: “Hangi feature türleri?” (kanal bazında)  
- **SA**: “Bu feature’lar nerede?” (konum bazında)

CA, konumu umursamaz (tüm H×W aynı ölçeklenir).  
SA, kanalı umursamaz (kanalları özetleyip konum maskesi çıkarır).

CBAM ikisini arka arkaya yapınca daha güçlü olur.


## 7) Neden CBAM bazen early layer’da iyi çalışmaz?

Early layer’larda çözünürlük yüksek (H×W büyük). Bu iki sonuç doğurur:

1) Spatial attention daha pahalı hale gelir (daha fazla piksel)  
2) Erken katman feature’ları çok temel olduğundan (kenar/gradient) “nereyi vurgulamalıyım” kararı daha belirsiz olabilir.

Bu yüzden pratikte CBAM çoğu zaman:
- Orta/derin stage’lerde daha net katkı verir.


## 8) Detection / Segmentation / Classification için pratik yerleştirme

**Classification**
- CBAM bazen faydalı, bazen gereksiz maliyet.
- En iyi ROI genelde orta/derin bloklarda.

**Detection**
- Head’de her yere koymak çoğu zaman gereksiz.
- Backbone veya neck içinde sınırlı kullanmak daha mantıklı.

**Segmentation**
- Spatial attention, sınır/region ayrımına yardım edebilir.
- Encoder’ın orta/derin katmanlarında denenir; decoder tarafında da seçici kullanım mantıklı olabilir.


## 9) Sık yapılan yanlış yorumlar

1) “CBAM nesneyi buluyor” → Hayır. CBAM “loss’u azaltan” pattern’leri öğreniyor.  
2) “CBAM piksel piksel sınıflandırma yapıyor” → Hayır. SA, maske üretip feature’ı ölçekliyor.  
3) “Mask 0/1 olur” → Hayır. Genelde yumuşak (soft) bir harita olur.


## 10) Minimal CBAM kodu (referans)

In [1]:
import torch
import torch.nn as nn

class ChannelAttention(nn.Module):
    def __init__(self, channels: int, reduction: int = 16, min_hidden: int = 4):
        super().__init__()
        hidden = max(min_hidden, channels // reduction)
        self.avg = nn.AdaptiveAvgPool2d(1)
        self.mx  = nn.AdaptiveMaxPool2d(1)
        self.mlp = nn.Sequential(
            nn.Conv2d(channels, hidden, 1, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, channels, 1, bias=True),
        )
        self.gate = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        w = self.gate(self.mlp(self.avg(x)) + self.mlp(self.mx(x)))
        return x * w

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size: int = 7):
        super().__init__()
        if kernel_size % 2 == 0:
            kernel_size += 1
        p = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=p, bias=False)
        self.gate = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        avg_map = torch.mean(x, dim=1, keepdim=True)
        max_map, _ = torch.max(x, dim=1, keepdim=True)
        s = torch.cat([avg_map, max_map], dim=1)
        w = self.gate(self.conv(s))
        return x * w

class CBAM(nn.Module):
    def __init__(self, channels: int, reduction: int = 16, sa_kernel: int = 7):
        super().__init__()
        self.ca = ChannelAttention(channels, reduction=reduction)
        self.sa = SpatialAttention(kernel_size=sa_kernel)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.ca(x)
        x = self.sa(x)
        return x


## 11) Hızlı test

In [2]:
x = torch.randn(2, 64, 56, 56)
m = CBAM(64, reduction=16, sa_kernel=7)
y = m(x)
x.shape, y.shape


(torch.Size([2, 64, 56, 56]), torch.Size([2, 64, 56, 56]))

---
---
---

# hidden ne?

* hidden, Channel Attention içindeki küçük MLP’nin ara kanal sayısıdır.

Bu satırda hesaplanıyor:
```python
hidden = max(min_hidden, channels // reduction)
```


Yani:

* Girdi kanal sayısı: channels = C

* Önce C → C/reduction ile sıkıştırılıyor

* Sonra tekrar C’ye açılıyor

Bu SE mantığıdır:

**Kanal bilgisini daha düşük boyutta özetle, sonra geri genişlet.***


# min_hidden ne?

* min_hidden, hidden’ın çok küçülmesini engelleyen alt sınırdır.

Örnek:

* channels = 16

* reduction = 16

* channels // reduction = 1

Eğer min_hidden olmasaydı:

* hidden = 1


Bu:

* Aşırı dar bir bottleneck

* Kanal ilişkilerini öğrenmek zor

* Attention “kör”leşir

min_hidden=4 sayesinde:
```python
hidden = max(4, 1) = 4
```

Kısaca özet
* hidden: kanal dikkatinin bilgi sıkıştırma boyutu

* min_hidden: bu boyutun aşırı küçülmesini önleyen güvenlik kemeri


## Tek cümlelik ezber

**hidden, channel attention’ın düşünme kapasitesi;**

**min_hidden, bu kapasitenin sıfıra düşmesini engeller.**

----
-----


## Neden 2 adet AdaptivePool2d (avg + max)?

* CBAM’in Channel Attention kısmı, kanalı “tek bir istatistikle” özetlemek istemiyor. Çünkü tek özet bazen sinyali kaçırır.

* AvgPool (ortalama): Kanalın genel olarak ne kadar aktif olduğunu yakalar.
Arka plan + nesne birlikte ortalamaya girer → “genel enerji” gibi düşün.

* MaxPool (maksimum): Kanalın bir yerde çok güçlü patladığı sinyali yakalar.
Nesne küçükse, ortalama düşük kalabilir ama max yüksek olur.

CBAM bu yüzden ikisini birlikte kullanır:
```python
mlp(avg(x)) + mlp(max(x))
```
Yani:

**“Genel aktivasyon” + “en güçlü aktivasyon” = daha sağlam kanal kararı.**



## Neden parametresi 1? (AdaptiveAvgPool2d(1))

AdaptiveAvgPool2d(1) demek:

* Çıkış uzamsal boyutu 1×1 olsun.

Yani:

* (B, C, H, W)  →  (B, C, 1, 1)


### Bu ne işe yarıyor?

* Her kanalı tek sayıya indiriyor.

* Çünkü channel attention kanal başına tek ağırlık üretmek istiyor.

* Sonra bu ağırlık tüm H×W’ye broadcast edilip kanalı çarpıyor.

Eğer 1 yerine 2 verseydik:

* (B, C, 2, 2) olurdu

* Bu artık “kanal attention” değil, kısmen “uzamsal” bilgi taşımaya başlar

* Tasarım amacı değişir (ve maliyet artar).

### Tek cümle özet

* 2 pool: Avg “genel” sinyal, Max “pik/tepe” sinyali yakalar → daha sağlam karar.

* 1: Kanalı tek skora indirip (C ağırlığı) tüm uzama uygulamak için.

----
----

Şu satırdaki:
```python
nn.Conv2d(channels, hidden, kernel_size=1)
```


* Bu Channel Attention içindeki MLP yerine kullanılan 1×1 convolution.

## kernel_size=1 ne anlama gelir?

1×1 convolution şunu yapar:

* Uzamsal boyuta hiç bakmaz

* Sadece kanallar arasında karışım (mixing) yapar

Yani:

* (B, C, 1, 1) → (B, hidden, 1, 1)


Her kanal:

* Aynı uzamsal noktada

* Diğer kanallarla lineer kombinasyon yapar

* Bu tam olarak Dense / Fully Connected davranışıdır.

## Neden 3×3 veya daha büyük değil?

* Çünkü bu noktada uzamsal bilgi zaten yok.

Hatırla:
```bash
x → AdaptiveAvgPool2d(1) → (B,C,1,1)
```


Artık:

* H = 1

* W = 1

Burada:

* 3×3 kernel anlamsız

* Zaten bakacak komşu piksel yok

* 1×1 tek mantıklı seçenektir.

### Asıl amaç ne?

Channel Attention şu soruyu sorar:

* “Bu kanal, diğer kanallara göre ne kadar önemli?”

Bu:

* Uzamsal değil

* Kanal ilişkisi problemidir

1×1 conv tam olarak şunu yapar:

* Kanal vektörünü al

* Daha küçük bir vektöre projekte et (bottleneck)

* Tekrar C boyutuna çıkar

**Bu = MLP**

## Neden Conv2d kullanıyoruz da Linear değil?

İki sebep:

### 1) Tensor şekli bozulmasın

* (B,C,1,1) 그대로 kalır

* view / flatten gerekmez

### 2) GPU / CNN uyumu

* Conv katmanları CNN graph’ında doğal

* Optimize kernel’lar var

* Kod daha temiz

Matematiksel olarak:
```python
Conv2d(kernel=1) ≡ Linear
```

## Kısa karşılaştırma

| Seçenek         | Neden uygun / değil           |
| --------------- | ----------------------------- |
| **1×1 Conv**    | ✅ Kanal karıştırma için ideal |
| 3×3 Conv        | ❌ Uzamsal bilgi yok           |
| Linear          | ⚠️ Çalışır ama reshape ister  |
| Attention (QKV) | ❌ Aşırı karmaşık              |


## Tek cümlelik ezber

Channel Attention’da kernel_size=1, çünkü burada yapılan şey uzamsal değil, kanal bazlı öğrenmedir.

----
----
---

Şu satırdan bahsediyoruz:
```python
self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=p, bias=False)
```

Yani:

* Giriş kanal sayısı = 2

* Çıkış kanal sayısı = 1

## Bu 2 kanal nereden geliyor?

SpatialAttention’da şu iki haritayı üretiyoruz:
```python
avg_map = torch.mean(x, dim=1, keepdim=True)   # (B,1,H,W)
max_map, _ = torch.max(x, dim=1, keepdim=True) # (B,1,H,W)
```


Bunlar:

* avg_map → “burada genel olarak aktivasyon var mı?”

* max_map → “burada çok güçlü bir sinyal var mı?”

Sonra:
```python
s = torch.cat([avg_map, max_map], dim=1)  # (B,2,H,W)
```


* İşte bu yüzden 2 kanal var.


## Peki neden 1 kanala düşürüyoruz?

* Çünkü spatial attention’ın çıktısı tek bir uzamsal maske olmalı.

Amaç:
```bash
(B, C, H, W)  ×  (B, 1, H, W)
```


Yani:

* Her konum (H,W) için tek bir “önem” değeri

* Tüm kanallar için aynı konumsal maske

* Eğer çıktı 2 kanal olsaydı:

* Hangi maskeyi kullanacaktın?

* Kanala özel spatial attention olurdu (tasarım değişir)

* CBAM’in tasarımı bu değil.

## 2 → 1 conv burada neyi öğreniyor?

Bu convolution şunu öğreniyor:

* “Avg ve Max uzamsal haritalarını nasıl birleştirirsem görev için daha iyi olur?”

Yani:

* Sadece toplamak değil

* Sadece ortalama almak değil

* Öğrenilebilir bir birleşim

Conv şunu yapar:

* Komşu piksellere bakar (kernel_size kadar)

* Avg ve Max’tan gelen bilgiyi bağlamsal olarak birleştirir

* Gürültüyü bastırır, yapısal bölgeleri öne çıkarır

## Neden 1×1 değil de 7×7 (veya 3×3)?

* Çünkü burada uzamsal bağlam istiyoruz.

* 1×1 → sadece aynı piksele bakar

* 7×7 → çevresine de bakar

Spatial attention’ın “neresi önemli?” demesi için:

**“Bu piksel tek başına mı önemli, yoksa çevresiyle birlikte mi anlamlı?”**

sorusuna cevap vermesi gerekir.

## Tek cümlelik ezber

**SpatialAttention’da 2→1 conv kullanıyoruz çünkü iki uzamsal ipucunu (avg & max) öğrenilebilir şekilde tek bir uzamsal maske haline getirmek istiyoruz.**

---
---

# CBAM neden CA → SA sırasını kullanır, tersini değil?
### Kısa cevap

**Çünkü önce “ne önemli”yi bilmeden “neresi önemli”yi sormak anlamsızdır.**

Uzun ama net açıklama

* Spatial Attention (SA) şunu yapar:

* Kanalları özetler

* Uzamsal bir maske üretir

* Ama bu özeti hangi kanallar üzerinden yapacağını bilmesi gerekir.

Eğer sıralama ters olsaydı:
```bash
x → Spatial Attention → Channel Attention
```

SA şu hatayı yapardı:

* Henüz gereksiz kanallar dururken

* Hepsini eşit kabul ederek

* Uzamsal maske üretirdi

Yani:

* “Gürültü + faydalı feature” birlikte spatial maskeye girer

* Bu da maskeyi bulanık ve kararsız yapar.

Doğru akış:
```bash
x → Channel Attention → (faydalı feature’lar kalır)
  → Spatial Attention → (bu faydalı feature’ların nerede önemli olduğu)
```

### 📌 Özet cümle:

* Önce kanıtları seç, sonra kanıtların yerini bul.

-----
----

## “Neden spatial attention kanala özel yapılmıyor?”

“Biz zaten Channel Attention yaparak kanal bazlı işlem yapmıyor muyuz?”

### 2.1 Kanal özel spatial attention ne olurdu?

Diyelim ki şöyle bir şey yaptık:
```bash
(B,C,H,W) → (B,C,H,W) mask
```


Yani:

* Her kanal için

* Ayrı bir uzamsal maske

#### Bu ne demek?

❌ Problemler

* Aşırı parametre

* Aşırı esneklik → overfitting

* Kanal–uzam ayrımı bulanıklaşır

**“Hangi feature nerede?” yerine , “Her şey her yerde” gibi bir yapı oluşur**

* Bu artık CBAM değil, bambaşka (ve ağır) bir attention olur.

### 2.2 CBAM’in bilinçli tercihi

CBAM şu ayrımı bilerek yapar:

```bash
Channel Attention → kanal seçimi

Spatial Attention → konum seçimi (kanallardan bağımsız)
```

Spatial attention’ın çıktısı:
```bash
(B,1,H,W)
```


Bu şu demektir:

**“Seçtiğim faydalı feature’lar için, bu konum genel olarak önemli mi?”**

Yani:

* Kanallar arası uzlaşma (consensus) aranır

* Tek bir kanalın aşırı patlaması değil

* Birden fazla kanalın desteklediği bölgeler öne çıkar

📌 Bu yüzden SA kanala özel yapılmaz.

---
---















































## CBAM neden Sigmoid kullanır?

* Bu soru çok önemli çünkü burada “attention = kapı (gate)” mantığı var.

#### 3.1 Sigmoid ne sağlar?

Sigmoid çıktısı:

* 0 ile 1 arasında


Bu şu anlama gelir:

* 0 → bastır

* 1 → tamamen geçir

* arası → yumuşak kontrol

CBAM’in yaptığı şey:
```bash
x * mask
```


**Bu bir ölçekleme işlemidir, seçim değil.**

#### 3.2 Neden Softmax değil?

Softmax:

* Rekabet yaratır

* Bir yer açılırken diğeri kapanmak zorunda kalır

Ama spatial attention’da:

* Birden fazla bölge aynı anda önemli olabilir

* Nesne + parça + bağlam birlikte çalışabilir

Sigmoid:

* Bağımsız karar verir

* Aynı anda birçok bölgeyi açık tutabilir

📌 Detection ve segmentation için bu çok kritik.

#### 3.3 Neden ReLU değil?

ReLU:

* Üst sınır yok

* Değerler şişebilir

* Attention “amplification”a dönüşür

* CBAM bunu istemez.
CBAM:

**“Dikkat = kontrol”,**
**“Güçlendirme = backbone’un işi”**